In [9]:
import os
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd

sys.path.append("../../../..")
from data.classification.toxic_comments.load import ToxicComtDataLoader

In [7]:
config_data = {
    "path_root": "../../../../data/classification/toxic_comments"
    , "path_data": "sample.csv"
    , "prob_train": 0.7
}

config_model = {
    "n_vocab": None
    , "dim_embed": 16
    , "idx_pad": None
    , "n_class": 2 

    , "dim_rnn_hidden": 8
    , "n_rnn_layer": 2
}

In [8]:

toxic_loader = ToxicComtDataLoader(
    config_data.get("path_root")
    , config_data.get("path_data")
    , config_data.get("prob_train")
)

config_model["n_vocab"] = len(toxic_loader.dataset.vocab)
config_model["idx_pad"] = len(toxic_loader.dataset.vocab.stoi.get("<PAD>"))

In [ ]:
class RNNLogisticRegres(nn.Module):

    def __init__(self, config):

        super().__init__()
        self.config = config

        self.embedding = nn.Embedding(
            num_embeddings=self.config.get("n_vocab")
            , embedding_dim=self.config.get("dim_embed")
            , padding_idx=self.config.get("idx_pad")
        )

        self.rnn = nn.RNN(
            input_size=self.config.get("dim_embed")
            , hidden_size=self.config.get("dim_rnn_hidden")
            , num_layers=self.config.get("n_rnn_layer")
            , batch_first=True
        )

        self.linear = nn.Linear(
            in_features=self.config.get("dim_rnn_hidden")
            , out_features=self.config.get("n_class")
            , bias=True
        )
        return

    def forward(self, x):

        batch_size = x.size(0)
        
        hidden = self.init_hidden(batch_size)
        out, hidden = self.rnn(x, hidden)
        
        out = out.contiguous().view(-1, self.config.get("dim_rnn_hidden"))
        out = self.linear(out)
        out = F.log_softmax(out, dim=-1)
        
        return out, hidden

    def init_hidden(self, batch_size):

        hidden = torch.zeros(
            self.config.get("n_rnn_layer")
            , batch_size
            , self.config.get("dim_rnn_hidden")
        )
        return hidden